In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import cv2
import os
import random

In [ ]:
input_img = cv2.imread('g:\Images\Original\image_1721814223451221704.png')
input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
plt.imshow(input_img)

In [ ]:
# def calculate_features(image):
#     # Create ORB detector
#     orb = cv2.ORB_create()

#     # Detect keypoints using ORB
#     keypoints = orb.detect(image, None)

#     # Compute descriptors
#     keypoints, descriptors = orb.compute(image, keypoints)

#     # Draw the detected keypoints on the image
#     output_image = cv2.drawKeypoints(image, keypoints, None, color=(0, 255, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

#     # Count the number of detected features
#     num_features = len(keypoints)
#     # print(f"Number of ORB features detected: {num_features}")

#     # Display the image with keypoints
#     # plt.imshow(output_image)
#     # plt.axis('off')
#     # plt.show()
    
#     return num_features, output_image

# features, image = calculate_features(input_img)
# print(features)
# plt.imshow(image)

In [ ]:
def calculate_features(image):

    fast12 = cv2.FastFeatureDetector_create(nonmaxSuppression = True)

    keypoints = fast12.detect(image, None)

    # Draw the detected keypoints on the image
    output_image = cv2.drawKeypoints(image, keypoints, None, color=(255, 0, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    # Count the number of detected features
    num_features = len(keypoints)
    # print(f"Number of FAST-12 features detected: {num_features}")

    # Display the image with keypoints
    # plt.imshow(output_image)
    return num_features, output_image

features, image = calculate_features(input_img)
print(features)
plt.imshow(image)


In [ ]:
# Define the list of image enhancement algorithms
image_enhancement_algorithms = ['WB','C_Up','C_Down','Bs_Up','B_Down','CLAHE']

# Define state space (number of features in intervals of hundreds)
state_space = ['F0','F1','F2','F3','F4','F5']

# # Define the environment (for simplicity, we assume a deterministic environment)
# def get_features(image, algorithm):
#     # Simulated function to compute the number of features
#     if algorithm == "Algorithm1":
#         return 2 * image  # Adjust this for different algorithms
#     elif algorithm == "Algorithm2":
#         return 3 * image
#     elif algorithm == "Algorithm3":
#         return 4 * image

In [ ]:
# Define the REINFORCE policy network
class PolicyNetwork(tf.keras.Model):
    def __init__(self, num_actions):
        super(PolicyNetwork, self).__init__()
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(num_actions, activation='softmax')

    def call(self, state):
        x = self.dense1(state)
        return self.dense2(x)

# Create the policy network and optimizer
policy_net = PolicyNetwork(len(image_enhancement_algorithms))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [ ]:
def CLAHE(image):

    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

    clahe_image = clahe.apply(gray_image)

    if len(image.shape) == 3:
        clahe_image = cv2.cvtColor(clahe_image, cv2.COLOR_GRAY2BGR)

    return clahe_image
image = CLAHE(input_img)
plt.imshow(image)


In [ ]:
def white_balance(image):
   
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    l, a, b = cv2.split(lab_image)

    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)

    balanced_lab_image = cv2.merge((cl, a, b))

    balanced_image = cv2.cvtColor(balanced_lab_image, cv2.COLOR_LAB2BGR)

    return balanced_image
image = white_balance(input_img)
plt.imshow(image)

In [ ]:
def Contrast_Up(image):
    
    contrasted_image = cv2.convertScaleAbs(image, alpha=1.25, beta=0)
    return contrasted_image
image = Contrast_Up(input_img)
plt.imshow(image)

In [ ]:
def Contrast_Down(image):
    
    contrasted_image = cv2.convertScaleAbs(image, alpha=0.75, beta=0)
    return contrasted_image
image = Contrast_Down(input_img)
plt.imshow(image)

In [ ]:
def Brightness_Up(image):
    
    brightened_image = cv2.convertScaleAbs(image, alpha=1.0, beta=15)
    return brightened_image
image = Brightness_Up(input_img)
plt.imshow(image)

In [ ]:
def Brightness_Down(image):
    
    darkened_image = cv2.convertScaleAbs(image, alpha=1.0, beta=-100)
    return darkened_image
image = Brightness_Down(input_img)
plt.imshow(image)

In [ ]:
def perform_action(ind,img_inp):
        if ind == 0:
            denoised = white_balance(img_inp)
            return denoised
        elif ind == 1:
            denoised = Contrast_Up(img_inp)
            return denoised
        elif ind == 2:
            denoised = Contrast_Down(img_inp)
            return denoised
        elif ind == 3:
            denoised = Brightness_Up(img_inp)
            return denoised
        elif ind == 4:
            denoised = Brightness_Down(img_inp)
            return denoised
        elif ind == 5:
            denoised = CLAHE(img_inp)
            return denoised

In [ ]:
def check_state(image):
        num_of_features, _ = calculate_features(image)
        if num_of_features < 0:
            return 'F0'
        elif num_of_features >= 0 and num_of_features < 100 :
            return 'F1'
        elif num_of_features <=200 and num_of_features > 100:
            return 'F2'
        elif num_of_features <=300 and num_of_features > 200:
            return 'F3'
        elif num_of_features <=400 and num_of_features > 300:
            return 'F4'
        elif  num_of_features > 400:
            return 'F5'

In [ ]:
def next_state(image ,action):
        next_img = perform_action(action,image)
        return [next_img,check_state(next_img)]

In [ ]:
def get_feature_value(ft1):
        i1 = state_space.index(ft1)
        return i1*100

In [ ]:
def update_reward(img1,img2):
        features_1, _ = calculate_features(img1)
        features_2, _ = calculate_features(img2)
        feature_difference = features_1 - features_2
        
        if feature_difference <0:
            return -5
        elif feature_difference == 0:
            return -1
        elif feature_difference <= 100 and feature_difference > 0:
            return 1
        elif feature_difference <= 200 and feature_difference > 100:
            return 2
        elif feature_difference <= 300 and feature_difference > 200:
            return 3
        elif feature_difference <= 400 and feature_difference > 300:
            return 4
        elif feature_difference > 400 :
            return 5

In [ ]:
# Define the training loop
def train_REINFORCE(num_episodes, discount_factor,input_img):
    cumulative_reward = 0
    
    for episode in range(num_episodes):
        state = np.random.choice(state_space)  # Random initial state
        state_tensor = tf.constant([[get_feature_value(state)]], dtype=tf.float32)

        curr_image = input_img

        with tf.GradientTape() as tape:
            action_probs = policy_net(state_tensor)
            action_distribution = tfp.distributions.Categorical(probs=action_probs)
            action = action_distribution.sample()
            action = int(action.numpy())

            if action == 0:
                nxt_state =  next_state(curr_image,0)
                next_state_tensor = tf.constant([[get_feature_value(nxt_state[1])]], dtype=tf.float32)
                den_img = nxt_state[0]
                reward = update_reward(den_img,curr_image)
                cumulative_reward += reward
                    
            elif action == 1:
                nxt_state =  next_state(curr_image,1)
                next_state_tensor = tf.constant([[get_feature_value(nxt_state[1])]], dtype=tf.float32)
                den_img = nxt_state[0]
                reward = update_reward(den_img,curr_image)
                cumulative_reward += reward
                    
            elif action == 2:
                nxt_state =  next_state(curr_image,2)
                next_state_tensor = tf.constant([[get_feature_value(nxt_state[1])]], dtype=tf.float32)
                den_img = nxt_state[0]
                reward = update_reward(den_img,curr_image)
                cumulative_reward += reward
                    
            elif action == 3:
                nxt_state =  next_state(curr_image,3)
                next_state_tensor = tf.constant([[get_feature_value(nxt_state[1])]], dtype=tf.float32)
                den_img = nxt_state[0]
                reward = update_reward(den_img,curr_image)
                cumulative_reward += reward
                    
            elif action == 4:
                nxt_state =  next_state(curr_image,4)
                next_state_tensor = tf.constant([[get_feature_value(nxt_state[1])]], dtype=tf.float32)
                den_img = nxt_state[0]
                reward = update_reward(den_img,curr_image)
                cumulative_reward += reward
                    
            elif action == 5:
                nxt_state =  next_state(curr_image,5)
                next_state_tensor = tf.constant([[get_feature_value(nxt_state[1])]], dtype=tf.float32)
                den_img = nxt_state[0]
                reward = update_reward(den_img,curr_image)
                cumulative_reward += reward
                    

            

            # Compute the loss
            loss = -tf.math.log(action_probs[0][action]) * reward

            curr_image = den_img
            end_ep_features, output_image = calculate_features(den_img)

        print(end_ep_features)
        plt.imshow(curr_image)
        # plt.imshow(output_image)

        # Compute gradients and update the policy network
        grads = tape.gradient(loss, policy_net.trainable_variables)
        optimizer.apply_gradients(zip(grads, policy_net.trainable_variables))

        # Print episode information
        print(f"Episode {episode + 1}: State={state}, Action={image_enhancement_algorithms[action]}, Reward={reward},Cumulative={cumulative_reward}")


In [ ]:
# Training parameters
num_episodes = 1000
discount_factor = 0.99

# Train the agent
train_REINFORCE(num_episodes, discount_factor,input_img)

In [ ]:
image = white_balance(input_img)
image = Contrast_Down(image)
image = Brightness_Up(image)
image = Brightness_Up(image)
features, image = calculate_features(image)
print(features)
plt.imshow(image)

In [ ]:
def enhance_underwater_image(img):
    # img = cv2.imread(img_path)
    # if img is None:
    #    print("Error: Image not found.")
    #    return

    # Convert image to RGB (OpenCV loads images in BGR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # White balancing using the Gray World assumption
    scale = img.mean(axis=(0, 1))
    img_balanced = img * (scale.mean() / scale)

    # Clip the values to [0, 255] and convert to uint8
    img_balanced = np.clip(img_balanced, 0, 255).astype(np.uint8)

    # Convert to LAB color space
    lab = cv2.cvtColor(img_balanced, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to L-channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))

    # Convert back to RGB color space
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)

    return enhanced_img
image = enhance_underwater_image(input_img)
plt.imshow(image)

In [ ]:
features, image = calculate_features(image)
print(features)
plt.imshow(image)